In [58]:
import json
import pandas as pd
import requests
from simple_salesforce import Salesforce, SalesforceLogin, SFType

In [59]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

In [60]:
results = sf.query_all(
    """
   select Id, IdentificationId__pc from Account 
    """
)
df = pd.DataFrame(results["records"])
df.shape
Pacientes_SF = df[["Id", "IdentificationId__pc"]]
Pacientes_SF.shape

KeyboardInterrupt: 

In [ ]:
# Recuperar empresas ID de salesforce
results = sf.query_all(
    """
   select Id, Name, Holding__c ,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,Empleador__c, ParentId, IsPersonAccount,poliza__c  from account where ispersonaccount = false 
    """
)
df = pd.DataFrame(results["records"])
df.shape
Empresas_account_SF = df[["Id", "Name", "Pa_s_del_Paciente__c", "Campa_a__c","AccountNumber",'Id_Empresa__c','Empleador__c','ParentId','IsPersonAccount','poliza__c']]
Empresas_account_SF.head()



In [ ]:
#traer empresas de accouint para unir con empresas sf
results = sf.query_all(
    """
   select id,Name,Nombre_Empresa__c,Empresa__c, Empresa_Padre__c from Programas_por_Empresas__c
    """
)
df = pd.DataFrame(results["records"])
df.shape
Empresas_SF = df[["Id", "Nombre_Empresa__c", "Name", "Empresa__c","Empresa_Padre__c"]]
Empresas_SF.head()


In [ ]:
empresas = pd.merge(
    left=Empresas_account_SF,
    right=Empresas_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
empresas.head()

In [ ]:
empresas.shape

In [ ]:
empresas.to_excel("F:/Empresas SF/reportes/emresas_.xlsx",index=False)

In [ ]:

# Relacion pacientes y empresas
results = sf.query_all(
    """
  select Id,Paciente__c,Programas_por_Empresas__c from Paciente_Programas__c 
    """
)
df = pd.DataFrame(results)
df.shape
Pacientes_empresas = pd.DataFrame(results["records"])
Pacientes_empresas = Pacientes_empresas[
    ["Paciente__c", "Programas_por_Empresas__c"]
]
Pacientes_empresas.head()

In [ ]:
#une Pacientes y el que hace de union Pacientes_empresas
pacientes_y_union = pd.merge(
    left=Pacientes_SF,
    right=Pacientes_empresas,
    how="left",
    left_on="Id",
    right_on="Paciente__c",
)
pacientes_y_union.head()

In [ ]:
pacientes_y_union.shape

In [ ]:
pacientes_y_empresas_final = pd.merge(
    left=pacientes_y_union,
    right=Empresas_SF,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Id",
)
pacientes_y_empresas_final.head()

In [ ]:
pacientes_y_empresas_final.shape

In [ ]:
nuevos = ("F:/Empresas SF/reportes/activos.xlsx")
Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].str.replace("-", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

In [ ]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
#Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.upper()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].astype(str)
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.strip()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["IdentificationId__pc"])

In [ ]:
#Pacientes_Archivo.head(20)

In [ ]:
pacientes_y_empresas_final["IdentificationId__pc"] = pacientes_y_empresas_final["IdentificationId__pc"].str.upper()
pacientes_y_empresas_final["IdentificationId__pc"] = pacientes_y_empresas_final["IdentificationId__pc"].astype(str)
pacientes_y_empresas_final["IdentificationId__pc"] = pacientes_y_empresas_final["IdentificationId__pc"].str.strip()
pacientes_y_empresas_final["IdentificationId__pc"] = pacientes_y_empresas_final["IdentificationId__pc"].str.replace(
    "-", ""
)
pacientes_y_empresas_final["IdentificationId__pc"] = pacientes_y_empresas_final["IdentificationId__pc"].str.replace(
    ".", ""
)

Pacientes_con_ID = pd.merge(
    left=Pacientes_Archivo,
    right=pacientes_y_empresas_final,
    how="left",
    left_on="rut",
    right_on="IdentificationId__pc",
)
Pacientes_con_ID.head()


In [ ]:
Pacientes_con_ID.to_excel(
    "F:/Empresas SF/reportes/Pacientes_y_empresas_nuevas.xlsx", index=False
)


In [ ]:
Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]
Final_pacientes_y_empresa = Final_pacientes_y_empresa[
    Final_pacientes_y_empresa["RutEstaoNo"] == False
]

if ("nombres" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["FirstName"] = Final_pacientes_y_empresa["nombres"]
if ("apellidos" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["LastName"] = Final_pacientes_y_empresa["apellidos"]
if ("rut" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Final_pacientes_y_empresa["rut"]
if ("email" in Final_pacientes_y_empresa.columns) == True:
    Final_pacientes_y_empresa["email"] = Final_pacientes_y_empresa["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = Final_pacientes_y_empresa["email"]
if ("nacimiento" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Final_pacientes_y_empresa["nacimiento"]
if ("sexo" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Final_pacientes_y_empresa["sexo"]
if ("direccion" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"].str.title().replace("NaN","")
    #Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Final_pacientes_y_empresa.columns) == True:
    #Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"].str.title().replace("NaN","")
    #Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Final_pacientes_y_empresa.columns) == True:
    #Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"].str.title().replace("NaN","") 
if ("cargo" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonTitle"] = Final_pacientes_y_empresa["cargo"]
if ("division" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonDepartment"] = Final_pacientes_y_empresa["division"]
if ("celular" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["Phone"] = Final_pacientes_y_empresa["celular"]


Subir_pacientes["BillingCountry"] = "Chile"
Subir_pacientes["Identification_Type__c"] = "RUT"
Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"

#Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel(
     "F:/Empresas SF/reportes/account.xlsx", index=False
)
Subir_pacientes.head()


In [ ]:
nuevos = ("F:/Empresas SF/reportes/activos.xlsx")
Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

pacientes_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="IdentificationId__pc",
)
pacientes_id.head()
pacientes_id.to_excel(
     "F:/Empresas SF/reportes/archivo_id.xlsx", index=False
)